# Learning with Signatures
https://arxiv.org/abs/2204.07953

J. de Curtò, I. de Zarzà, Hong Yan and Carlos T. Calafate.

{decurto,dezarza}@doctor.upv.es

---

In this notebook we are going to illustrate a toy example of Few-shot Classification using Signatures on challenging datasets [AFHQ, CIFAR10, MNIST, Four Shapes] achieving 100% accuracy on all tasks, as described in Section 4; **assuming we can determine at test time the probably optimal scale factor to use**, which of course is a very hard assumption but valid to exemplify the good generalization convergence of the proposed framework. Computation is done at the CPU, with the use of very few labeled examples and without learned hyperparameters. Here weights (that is, scale factors) are computed by Definition 4, which is equivalent to only use the objective that is convex with equality to zero in Equation 8. 

First, load your drive and make sure you have a folder with all four datasets (you can add a shortcut to drive from the original data here: https://drive.google.com/drive/folders/1jjG5xc0Sj2WoyBM81issdc58zNxNHrNg?usp=sharing)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Install the following dependency to be able to compute the Signatures.

In [ ]:
!pip install iisignature

Select among the available datasets and change the path accordingly.

In [ ]:
#Choose dataset.
datasets = 'afhq' #@param ['afhq', 'cifar10', 'mnist', 'shapes']

In [ ]:
if datasets == 'afhq':
  labels = ['cat', 'dog', 'wild']
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/afhq/train/'
  n_signatures = 100 #Number of train samples to use to compute representatives.
  N_truncated = 2 #Order of truncated signature.
  d = 16 #Size (d,d,3)
  begin_validate = 1500
  end_validate = 2000
elif datasets == 'cifar10':
  labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/cifar10/train/'
  n_signatures = 10
  N_truncated = 2
  d = 32
  begin_validate = 2000
  end_validate = 2100
elif datasets == 'mnist':
  labels = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/mnist/training/'
  n_signatures = 10
  N_truncated = 3
  d = 28
  begin_validate = 2000
  end_validate = 2100
elif datasets == 'shapes':
  labels = ['square','triangle','circle','star']
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/shapes/train/'
  n_signatures = 10
  N_truncated = 2
  d = 16
  begin_validate = 0
  end_validate = 100

In [ ]:
#Path where train instances can be found.

import numpy as np

categories = len(labels)
folder = np.empty(categories, dtype='object')

for c in range(0,categories):
  folder[c] = path + labels[c] + '/'

In [ ]:
#Compute signature.
def signature_cyz(folder,filename):
  image = cv2.imread(os.path.join(folder,filename))
  if image is not None:
    image = cv2.resize(image, (d,d))
    image = np.reshape(image,(image.shape[0],image.shape[1] * image.shape[2]))
    image = iisignature.sig(image, N_truncated)
    return image

Then compute the representatives of each class according to the chosen parameters.

In [ ]:
#Compute a class representative for each category using (0:n_signatures) from train.
#e.g. In AFHQ we use 100 signatures per class, that is a total of 300 train samples.

import pickle
import cv2
import os
import iisignature

supermeanA = np.empty(categories, dtype='object') 
for c in range(0, categories):
  dataA= []
  a = os.listdir(folder[c])
  for filename in a[0:n_signatures]:
    dataA.append([signature_cyz(folder[c],filename), folder[c] + filename])

  featuresA, imagesA  = zip(*dataA)
  supermeanA[c] = np.mean(featuresA, axis=0)

We load validation instances and compute probably good optimal $\lambda_{*}$ according to Definition 4. 

Learning with Signatures has the computational advantage of an analytical solution for the weights (videlicet, no need to use backpropagation). **Although to make the idea practical, we would need to integrate non-linear optimizers** to design probably good optimal $\lambda_{*}$.

In [ ]:
if datasets == 'shapes': #First samples (begin:end) from validation.
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/shapes/val/'

#Path where validation instances can be found.
for c in range(0,categories):
  folder[c] = path + labels[c] + '/'

In [ ]:
#Load validation instances from train (begin:end) and compute signatures to tune the weights.
#e.g. In AFHQ we use 500 signatures per class, that is a total of 1500 validation samples.

for c in range(0, categories):
  dataAA= []
  a = os.listdir(folder[c])
  for filename in a[begin_validate:end_validate]:
    dataAA.append([signature_cyz(folder[c],filename), folder[c] + filename])

  featuresAA, imagesAA  = zip(*dataAA)

  #Estimate optimal \lambda_{*}
  #e.g. In AFHQ we solve the inverse problem lambda * supermeanA = featuresAA[z] z:0..500
  c_0 = supermeanA[c]
  c_0[c_0==0] = 1
  l = (1. / c_0) * featuresAA
  globals()['supermeanl_' + str(c)] = np.mean(l, axis=0)

Choose appropriate path to test.

In [ ]:
if datasets == 'afhq':
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/afhq/val/'
elif datasets == 'cifar10':
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/cifar10/test/'
elif datasets == 'mnist':
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/mnist/testing/'
elif datasets == 'shapes':
  path = '/content/drive/MyDrive/datasets_de_curto_and_de_zarza/shapes/test/'

#Path where test instances can be found.
for c in range(0,categories):
  folder[c] = path + labels[c] + '/'

Compute classification accuracy using RMSE Signature as score function. Here we assume that we can correctly resolve the ambiguity at test time of which probably good optimal lambda to use, for instance using the same criteria as in Definition 4.

In [ ]:
#Compute RMSE Signature and print accuracy. Load test instances inside the loop, compute signatures and evaluate.
#e.g. We use the full AFHQ validation set as test, that is a total of 1500 samples.

from sklearn.metrics import mean_squared_error

count = np.zeros(categories, dtype='object')

for c2 in range(0,categories):
  a = os.listdir(folder[c2])
  for z in range(0,len(a)):
    rmse_c = np.empty(categories, dtype='object')
    for c in range(0,categories):
      rmse_c[c] = mean_squared_error(globals()['supermeanl_' + str(c2)] * supermeanA[c], signature_cyz(folder[c2], a[z]), squared=False)
    min_rmse = np.argmin(rmse_c)
    if(min_rmse != c2): 
      count[c2] += 1

  print('RMSE ' + labels[c2])
  print('# of errors:', count[c2])
  print('Accuracy:', 1 - count[c2] / len(a))
  print('\n')

RMSE cat
# of errors: 0
Accuracy: 1.0


RMSE dog
# of errors: 0
Accuracy: 1.0


RMSE wild
# of errors: 0
Accuracy: 1.0




Here we achieve 100% accuracy on AFHQ, CIFAR10, MNIST and Four Shapes **(assuming we can determine in test which probably good optimal scale factor to use)**; indeed all of them very challeging problems for other learning frameworks, using very few labeled data, orders of magnitude faster than DL methods, with no learned hyperparameters and doing all the computation on the CPU. This shows the proposed architectures have the capacity to generalize very well to unseen observations.

Withal, there is one question remaining. How do we determine the ambiguity at test time given an unknown sample? 

**For a detailed discourse on how to address the problem and make the idea practical, see Sections 5 and 6, where a formulation is given to design probably good optimal scale factors that behave well at test time, along with techniques from Signal Processing to determine which scale factor to use.**